<a href="https://colab.research.google.com/github/CodeAlgorilla/SpeedyInference/blob/feature%2Fcolab_notebook/early_exit_self_speculative_decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by: [Mostafa Elhoushi](https://huggingface.co/melhoushi)

## Install requirements

First, run the cells below to install the requirements:

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

## Setup

Let's load a couple of Llama models.

To access the Llama models you need:
1. Submit a request to access the [Llama3](https://huggingface.co/meta-llama/Meta-Llama-3-8B) and [LayerSkip](https://huggingface.co/facebook/layerskip-llama3-8B) checkpoints.
2. Setup 🤗 token access by following the steps described [here](https://huggingface.co/docs/hub/en/security-tokens).

Then you will be able to login to 🤗 by running the cell below and entering the token you obtain from Step 2 above:

In [ ]:
!huggingface-cli login

Now, let's import some libraries and classes that we will need in this demo.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 500)

print function for all outputs at each layer

In [ ]:
def get_early_exit_predictions(hidden_states, lm_head, tokenizer):
  layer2text = dict()
  device = hidden_states[0][0].device
  for layer_idx in range(len(hidden_states[0])):
    output_ids = torch.empty((1,1), device=device, dtype=torch.int)
    for token_idx in range(len(hidden_states)):
      logits = lm_head(hidden_states[token_idx][layer_idx])
      probs = torch.nn.functional.softmax(logits, dim=-1)
      out = torch.argmax(probs, dim=-1)
      output_ids = torch.cat((output_ids, out), dim=-1)
    text = tokenizer.batch_decode(output_ids[0])
    layer2text[layer_idx] = text
  return layer2text

And let's set some default generation configuration for this demo:

In [ ]:
generation_config = {
    "do_sample": False,
    "temperature": None,
    "top_p": None,
    "max_new_tokens": 100
}

Choose the origin model ad the layerskip model

In [ ]:
orig_checkpoint = "meta-llama/Meta-Llama-3-8B"         # meta-llama/Llama-2-7b-hf       meta-llama/Llama-2-13b-hf       meta-llama/CodeLlama-7b-hf        meta-llama/Meta-Llama-3-8B     meta-llama/Llama-3.2-1B 
layerskip_checkpoint = "facebook/layerskip-llama3-8B"  # facebook/layerskip-llama2-7B   facebook/layerskip-llama2-13B   facebook/layerskip-codellama-7B   facebook/layerskip-llama3-8B   facebook/layerskip-llama3.2-1B

Load original model

In [ ]:
orig_model = AutoModelForCausalLM.from_pretrained(
    orig_checkpoint,
    torch_dtype=torch.float16,
    device_map="auto",
)
orig_tokenizer = AutoTokenizer.from_pretrained(
    orig_checkpoint
)

orig_model.generation_config.pad_token_id = orig_tokenizer.eos_token_id

Load layerskip model

In [ ]:
layerskip_model = AutoModelForCausalLM.from_pretrained(
    layerskip_checkpoint,
    torch_dtype=torch.float16,
    device_map="auto",
)
layerskip_tokenizer = AutoTokenizer.from_pretrained(
    layerskip_checkpoint
)

layerskip_model.generation_config.pad_token_id = layerskip_tokenizer.eos_token_id

Let's create a prompt:

In [ ]:
prompt = "Once upon a time"
code_prompt = ""

Generate tokens using origin model

In [ ]:
orig_inputs = orig_tokenizer(prompt, return_tensors="pt").to(orig_model.device)


orig_outputs = orig_model.generate(
    **orig_inputs,
    **generation_config,
    return_dict_in_generate=True,
    output_hidden_states=True
)

# Convert Output Token IDs to Output Text
orig_text = orig_tokenizer.decode(orig_outputs[0], skip_special_tokens=True)
print(orig_text)

In [ ]:

# check hidden states
hidden_states = orig_outputs["hidden_states"]
print("hidden_states:")

print(f"  type(hidden_states): {type(hidden_states)}")
print(f"  len(hidden_states): {len(hidden_states)}")

print(f"    type(hidden_states[0]): {type(hidden_states[0])}")
print(f"    len(hidden_states[0]): {len(hidden_states[0])}")

print(f"      type(hidden_states[0][0]): {type(hidden_states[0][0])}")
print(f"      hidden_states[0][0].shape: {hidden_states[0][0].shape}")
print(f"      hidden_states[1][0].shape: {hidden_states[1][0].shape}")

batch_size, input_seq_len = orig_inputs["input_ids"].shape
batch_size, total_seq_len = orig_outputs["sequences"].shape

prompt_len = input_seq_len
num_steps = total_seq_len - input_seq_len
emb_dim = orig_model.config.hidden_size
num_layers = len(orig_model.model.layers)

print(f"batch_size: {batch_size}\n"
      f"prompt_len: {prompt_len}\n"
      f"num_steps: {num_steps}\n"
      f"emb_dim: {emb_dim}\n"
      f"num_layers: {num_layers}")

assert(len(hidden_states) == num_steps)
assert(len(hidden_states[0]) == num_layers + 1) # add 1 to count embedding layer
# Tensors of step 0 process prompt
assert(hidden_states[0][0].shape == (batch_size, prompt_len, emb_dim))
# Tensors of each remaining step processes a single token
assert(hidden_states[1][0].shape == (batch_size, 1, emb_dim))

Now, let's print a table that shows the full predicted text when exiting at each layer in origin model:

In [ ]:
orig_layer_2_text = get_early_exit_predictions(
    orig_outputs["hidden_states"],
    orig_model.lm_head,
    orig_tokenizer
)

orig_df = pd.DataFrame.from_dict(orig_layer_2_text, orient="index", columns=np.arange(total_seq_len))
orig_df = orig_df.style.set_properties(**{'text-align': 'left'})

In [ ]:
orig_df

Show the full predicted text when exiting at each layer in a layerskip model

In [ ]:
layerskip_outputs = layerskip_model.generate(
    **orig_inputs.to(layerskip_model.device),
    **generation_config,
    return_dict_in_generate=True,
    output_hidden_states=True
)

layerskip_layer_2_text = get_early_exit_predictions(
    layerskip_outputs["hidden_states"],
    layerskip_model.lm_head,
    layerskip_tokenizer
)

layerskip_text = layerskip_tokenizer.decode(layerskip_outputs["sequences"][0], skip_special_tokens=True)
print(layerskip_text)

In [ ]:
layerskip_df = pd.DataFrame.from_dict(layerskip_layer_2_text, orient="index", columns=np.arange(total_seq_len))
layerskip_df = layerskip_df.style.set_properties(**{'text-align': 'left'})

In [ ]:
layerskip_df